In [3]:
from gensim import corpora, models, similarities, matutils, utils
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import PCA
from sklearn.cluster import k_means
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import metrics
from nltk import FreqDist
import pandas as pd
import numpy as np
import time
import nltk
import pdb
import re
import os

C:\Users\Adi\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
tokenizer = RegexpTokenizer("\w+")
stop_words=set(stopwords.words('english'))

# DO NOT RUN THIS PART

In [ ]:
subd=pd.read_hdf('Sub_data.h5', 'subs')
movd=pd.read_hdf('movbaseB.h5', 'imdb_id')

In [ ]:
print('Number subs in database: {}'.format(len(subd.columns.levels[0])))
print('Here are the shitty ones:')
t0 = time.time()
for item in subd.columns.levels[0]:
    n_lines=len(subd[item][subd[item]['Line'] != ''])  #number of lines
    num_s=[item2 in ' '.join(subd[item]['Line']).strip().lower() for item2 in stop_words].count(True) #number of stop words
    if n_lines < 100 or num_s < 100:
        print('{}-{}-{} :: '.format(movd[movd.imdb_id == item]['Title'].values[0],item,num_s),end=' ')
        subd.drop(item,axis=1,level=0,inplace=True)
        subd.columns = pd.MultiIndex.from_tuples(subd.columns.to_series())
t1 = time.time()
print('Number subs after removel of shitty ones: {}'.format(len(subd.columns.levels[0])))        
print('Time elapsed cleaning: {}'.format(t1-t0))

In [ ]:
subd.to_hdf('Sub_data_cleaned.h5', 'subs', mode='w')

In [ ]:
searchw=['xb6 '] #search for certain words in the text
for item in subd.columns.levels[0]:
    num_s=[item2 in ' '.join(subd[item]['Line']).strip().lower() for item2 in searchw].count(True)
    if num_s > 0:
        print('{}-{}-{}'.format(movd[movd.imdb_id == item]['Title'].values[0],item,num_s))

# Start from here if you dont want to clean the data

In [5]:
subd=pd.read_hdf('Sub_data_cleaned.h5', 'subs')
movd=pd.read_hdf('movbaseB.h5', 'imdb_id')
movd.drop_duplicates(subset='imdb_id',inplace=True)

# Clean text

In [12]:
raw={}  #dictionary with imdb_id as keys and string of entire script (uncleaned)
mtok={} #dictionary with imdb_id as keys and tokenized list of words for script
mltk={} #dictionary with imdb_id as keys and nltk.text.Text
fdist={} #dictionary with imdb_id as keys and nltk.probability.FreqDist

for item in subd.columns.levels[0]:
        raw[item]=" ".join(subd[item]['Line']).strip().lower()
        raw[item]=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', raw[item]) #removes web addressses
        raw[item]=re.sub('<[^>]+>', '', raw[item]) #removes < crap > 
        raw[item]=re.sub('xe2\w+', '', raw[item]) #removes weird shit like xe2x99xaa
        raw[item]=re.sub('xc2\w+', '', raw[item]) #removes weird shit like xe2x99xaa
        raw[item]=re.sub('x9.*? ', '', raw[item]) #removes weird shit like x92 
        raw[item]=re.sub('xa7.*? ', '', raw[item])
        mtok[item]=[w for w in tokenizer.tokenize(raw[item]) if not w in stop_words]
        mltk[item]=nltk.Text([w for w in tokenizer.tokenize(raw[item]) if not w in stop_words])
        fdist[item]=FreqDist(mltk[item])

In [13]:
alls=' '.join(list(raw.values()))
allst=[item for items in list(mtok.values()) for item in items]
print('Total word count:{}'.format(len(alls.split())))
print('Total word count after stop word removal:{}'.format(len(allst)))

Total word count:13098347
Total word count after stop word removal:6264173


In [6]:
for key, value in raw.items():
    print('Imdb_id: {}  Length: {} Genre: {} Title: {}'
          .format(key,len(value), movd.loc[movd['imdb_id'] == key, 'Genre'].values[0]
                  , movd.loc[movd['imdb_id'] == key, 'Title'].values[0]))

Imdb_id: 0029583  Length: 21654 Genre: Family Title: Snow White and the Seven Dwarfs
Imdb_id: 0031381  Length: 137449 Genre: Romance Title: Gone with the Wind
Imdb_id: 0032455  Length: 10143 Genre: Music Title: Fantasia
Imdb_id: 0032910  Length: 33326 Genre: Family Title: Pinocchio
Imdb_id: 0034492  Length: 10544 Genre: Family Title: Bambi
Imdb_id: 0042332  Length: 30633 Genre: Family Title: Cinderella
Imdb_id: 0043274  Length: 37868 Genre: Family Title: Alice in Wonderland
Imdb_id: 0046183  Length: 38275 Genre: Music Title: Peter Pan
Imdb_id: 0048280  Length: 28065 Genre: Family Title: Lady and the Tramp
Imdb_id: 0052618  Length: 36612 Genre: Drama Title: Ben-Hur
Imdb_id: 0055254  Length: 32666 Genre: Family Title: One Hundred and One Dalmatians
Imdb_id: 0055614  Length: 53013 Genre: Music Title: West Side Story
Imdb_id: 0058331  Length: 70170 Genre: Family Title: Mary Poppins
Imdb_id: 0058385  Length: 94000 Genre: Music Title: My Fair Lady
Imdb_id: 0059742  Length: 82370 Genre: Music

Imdb_id: 0118884  Length: 64583 Genre: Drama Title: Contact
Imdb_id: 0118971  Length: 70057 Genre: Horror Title: The Devil's Advocate
Imdb_id: 0118972  Length: 31199 Genre: Drama Title: The Devil's Own
Imdb_id: 0118998  Length: 51658 Genre: Family Title: Doctor Dolittle
Imdb_id: 0119137  Length: 33581 Genre: Family Title: Flubber
Imdb_id: 0119217  Length: 84103 Genre: Drama Title: Good Will Hunting
Imdb_id: 0119282  Length: 43105 Genre: Family Title: Hercules
Imdb_id: 0119303  Length: 26022 Genre: Family Title: Home Alone 3
Imdb_id: 0119314  Length: 58823 Genre: Romance Title: The Horse Whisperer
Imdb_id: 0119345  Length: 23031 Genre: Horror Title: I Know What You Did Last Summer
Imdb_id: 0119528  Length: 42564 Genre: Comedy Title: Liar Liar
Imdb_id: 0119643  Length: 72499 Genre: Drama Title: Meet Joe Black
Imdb_id: 0119654  Length: 36413 Genre: Action Title: Men in Black
Imdb_id: 0119715  Length: 34202 Genre: Family Title: MouseHunt
Imdb_id: 0119738  Length: 32271 Genre: Comedy Title:

Imdb_id: 0329575  Length: 47391 Genre: Drama Title: Seabiscuit
Imdb_id: 0330373  Length: 52778 Genre: Family Title: Harry Potter and the Goblet of Fire
Imdb_id: 0332280  Length: 50920 Genre: Romance Title: The Notebook
Imdb_id: 0332379  Length: 61500 Genre: Music Title: School of Rock
Imdb_id: 0332452  Length: 45696 Genre: Drama Title: Troy
Imdb_id: 0335345  Length: 14678 Genre: Drama Title: The Passion of the Christ
Imdb_id: 0337741  Length: 57971 Genre: Comedy Title: Something's Gotta Give
Imdb_id: 0337851  Length: 50769 Genre: Drama Title: Baywatch: The Reunion
Imdb_id: 0338094  Length: 45139 Genre: Family Title: The Haunted Mansion
Imdb_id: 0338348  Length: 25735 Genre: Family Title: The Polar Express
Imdb_id: 0338459  Length: 32476 Genre: Action Title: Spy Kids 3-D: Game Over
Imdb_id: 0338526  Length: 39846 Genre: Action Title: Van Helsing
Imdb_id: 0339291  Length: 48988 Genre: Family Title: Lemony Snicket's A Series of Unfortunate Events
Imdb_id: 0343660  Length: 50011 Genre: Com

Imdb_id: 0892782  Length: 36329 Genre: Family Title: Monsters vs Aliens
Imdb_id: 0892791  Length: 37056 Genre: Family Title: Shrek Forever After
Imdb_id: 0903624  Length: 45843 Genre: Action Title: The Hobbit: An Unexpected Journey
Imdb_id: 0910970  Length: 18957 Genre: Family Title: WALL·E
Imdb_id: 0926084  Length: 49669 Genre: Family Title: Harry Potter and the Deathly Hallows: Part 1
Imdb_id: 0938283  Length: 27386 Genre: Action Title: The Last Airbender
Imdb_id: 0948470  Length: 42878 Genre: Action Title: The Amazing Spider-Man
Imdb_id: 0952640  Length: 39891 Genre: Music Title: Alvin and the Chipmunks
Imdb_id: 0960731  Length: 60307 Genre: Family Title: Bedtime Stories
Imdb_id: 0963966  Length: 38504 Genre: Action Title: The Sorcerer's Apprentice
Imdb_id: 0970179  Length: 31920 Genre: Family Title: Hugo
Imdb_id: 0970416  Length: 35032 Genre: Drama Title: The Day the Earth Stood Still
Imdb_id: 0970866  Length: 72152 Genre: Comedy Title: Little Fockers
Imdb_id: 0976238  Length: 5203

Imdb_id: 1843866  Length: 47288 Genre: Action Title: Captain America: The Winter Soldier
Imdb_id: 1853728  Length: 69535 Genre: Drama Title: Django Unchained
Imdb_id: 1854564  Length: 41692 Genre: Family Title: Percy Jackson: Sea of Monsters
Imdb_id: 1855325  Length: 15890 Genre: Action Title: Resident Evil: Retribution
Imdb_id: 1860353  Length: 33869 Genre: Family Title: Turbo
Imdb_id: 1872181  Length: 59681 Genre: Action Title: The Amazing Spider-Man 2
Imdb_id: 1877832  Length: 38656 Genre: Action Title: X-Men: Days of Future Past
Imdb_id: 1905041  Length: 43513 Genre: Action Title: Fast & Furious 6
Imdb_id: 1907668  Length: 69275 Genre: Drama Title: Flight
Imdb_id: 1911658  Length: 39052 Genre: Family Title: Penguins of Madagascar
Imdb_id: 1915581  Length: 60925 Genre: Drama Title: Magic Mike
Imdb_id: 1921064  Length: 28154 Genre: Romance Title: Pompeii
Imdb_id: 1951261  Length: 45730 Genre: Comedy Title: The Hangover Part III
Imdb_id: 1951264  Length: 47556 Genre: Action Title: The

# LDA model

In [14]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation



data_samples=list(raw.values())

n_samples = len(data_samples)
n_features = 2000
n_components = 40
n_top_words = 30

#words occurring in
#only one document or in at least 95% of the documents are removed.


# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.6, min_df=5,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)

terms=tf_vectorizer.get_feature_names() #all terms
print("done in %0.3fs." % (time() - t0))
print()

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=10,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
X=lda.fit_transform(tf)
print("done in %0.3fs." % (time() - t0))



def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)



# #############################################################################
# Do the actual clustering
num_clusters=20

km = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100, n_init=1)


print("Clustering sparse data with %s" % km)
t0 = time()
pred=km.fit_predict(X)
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()



Extracting tf features for LDA...
done in 9.118s.

Fitting LDA models with tf features, n_samples=1570 and n_features=2000...
done in 157.370s.

Topics in LDA model:
Topic #0: john dave joe alice jimmy simon dog howard ryan sarah matt mom brian honey girls dogs dinner party married june phil jordan jim drink dream clark wow bed sweet april
Topic #1: captain ship planet earth space copy mission control team air roger plane power flight seconds crew launch elliot fly pull target signal pilot commander lieutenant computer base moving weapons force
Topic #2: em ya fight ain doin goin outta race somethin rocky nothin win comin talkin yo speed lookin corn gettin uh round beat champion rock cars ring brian eye mon corner
Topic #3: sherman professor scarlett penny ain jason buddy captain ass mrs ma mama war dear fat child dr marry gentlemen sick hurry lord daddy folks ladies stone party glad paul evening
Topic #4: dr lucy doctor evil vampire blood van ed powers scott fat accident priest bastar

In [15]:
from sklearn.manifold import TSNE
tsne_model=TSNE(n_components=2,verbose=1,random_state=0,angle=0.99,init='pca')
tsne_lda=tsne_model.fit_transform(X)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1570 samples in 0.005s...
[t-SNE] Computed neighbors for 1570 samples in 0.156s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1570
[t-SNE] Computed conditional probabilities for sample 1570 / 1570
[t-SNE] Mean sigma: 0.139021
[t-SNE] KL divergence after 250 iterations with early exaggeration: 73.261658
[t-SNE] Error after 1000 iterations: 1.183052


In [16]:
import numpy as np
import bokeh.plotting as bp
from bokeh.plotting import save
from bokeh.models import HoverTool

n_top_words = 5 # number of keywords we show



In [18]:
title = 'Stuff'
num_example = len(X)
_lda_keys = []
# 20 colors
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
])

for i in range(0,X.shape[0]):
  _lda_keys +=  X[i].argmax(),
plot_lda = bp.figure(plot_width=1400, plot_height=1100,
                     title=title,
                     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x=tsne_lda[:, 0], y=tsne_lda[:, 1],
                 color=colormap[_lda_keys][:num_example])
save(plot_lda, '{}.html'.format(title))

C:\Users\Adi\Anaconda3\lib\site-packages\bokeh\io.py:527: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warnings.warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\Users\Adi\Anaconda3\lib\site-packages\bokeh\io.py:537: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warnings.warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'C:\\bArxiv\\GoogleDrive\\GLibrary\\P.Python\\FilmProject\\Stuff.html'

In [20]:
from collections import defaultdict

g1=[]
groups=defaultdict.fromkeys(np.unique(pred),[])
for i in range(0,pred.size):
    tid=subd.columns.levels[0][i]
    title=movd[movd.imdb_id == tid]['Title'].values[0]
    if pred[i] == 3:
        g1.append(title)
g1

['Return of the Jedi',
 'Pocahontas',
 'The Lord of the Rings: The Fellowship of the Ring',
 'Planet of the Apes',
 'Spirit: Stallion of the Cimarron',
 'The Lord of the Rings: The Return of the King',
 'The Lord of the Rings: The Two Towers',
 'Wolves',
 'The Jungle Book 2',
 'Troy',
 '300',
 'Kung Fu Panda',
 'Beowulf',
 '10,000 BC',
 'Puss in Boots',
 'Eragon',
 'TMNT',
 'The Croods',
 'The Chronicles of Narnia: Prince Caspian',
 'Oceans',
 'Clash of the Titans',
 'Warcraft',
 'Dracula Untold',
 'How to Train Your Dragon',
 'The Hobbit: An Unexpected Journey',
 'The Last Airbender',
 'Silent Hill: Revelation 3D',
 'Turtle: The Incredible Journey',
 'Harry Potter and the Deathly Hallows: Part 2',
 'Doctor Strange',
 "Legend of the Guardians: The Owls of Ga'Hoole",
 'African Cats',
 'Immortals',
 'Hercules',
 'Kung Fu Panda 2',
 '47 Ronin',
 'Hansel & Gretel: Witch Hunters',
 'The Host',
 'Maleficent',
 'Oz: The Great and Powerful',
 'Cave of Forgotten Dreams',
 'Born to Be Wild',
 'S